In [9]:
import utils
import torch
import transforms as T
import torchvision

from tqdm import tqdm
from engine import train_one_epoch, evaluate
from tars.base.dataset import DatasetType
from tars.datasets.instance_segmentation_dataset import InstanceSegmentationDataset
from torchvision.models.detection import maskrcnn_resnet50_fpn


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [6]:
split = DatasetType.VAL_UNSEEN
dataset = InstanceSegmentationDataset(split, transforms=get_transform(train=False))
# indices = torch.randperm(256).tolist()
# dataset = torch.utils.data.Subset(dataset, indices)

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=16, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [7]:
len(dataset)

14418

In [10]:
for i in tqdm(range(len(dataset))):
    img, tgt = dataset[i]
    assert(tgt != None)
    assert(tgt["labels"].shape[0] > 0)
    assert(tgt["labels"].shape[0] == tgt["boxes"].shape[0])
    assert(tgt["boxes"].shape[0] == tgt["masks"].shape[0])

100%|██████████| 14418/14418 [10:03<00:00, 23.89it/s]


In [11]:
device = torch.device('cuda:2')
maskrcnn = maskrcnn_resnet50_fpn(num_classes=119)
maskrcnn.eval()
maskrcnn.load_state_dict(torch.load('../moca/weight_maskrcnn.pt'))
maskrcnn = maskrcnn.to(device)

In [12]:
results = evaluate(maskrcnn, data_loader, device=device)

100%|██████████| 14418/14418 [12:18<00:00, 19.53it/s]
creating index...
index created!
Test:  [  0/902]  eta: 2:53:11  model_time: 6.1048 (6.1048)  evaluator_time: 1.6024 (1.6024)  time: 11.5207  data: 2.4327  max mem: 174
Test:  [100/902]  eta: 0:35:51  model_time: 1.3293 (1.5677)  evaluator_time: 0.7518 (0.9722)  time: 2.2987  data: 0.0437  max mem: 174
Test:  [200/902]  eta: 0:30:52  model_time: 1.5477 (1.5486)  evaluator_time: 0.9313 (0.9665)  time: 2.5630  data: 0.0440  max mem: 174
Test:  [300/902]  eta: 0:26:21  model_time: 1.5693 (1.5431)  evaluator_time: 0.9378 (0.9631)  time: 2.5215  data: 0.0431  max mem: 174
Test:  [400/902]  eta: 0:22:04  model_time: 1.5698 (1.5510)  evaluator_time: 0.9487 (0.9704)  time: 2.6030  data: 0.0434  max mem: 174
Test:  [500/902]  eta: 0:18:11  model_time: 1.3348 (1.6039)  evaluator_time: 0.7307 (0.9862)  time: 2.3753  data: 0.0435  max mem: 174
Test:  [600/902]  eta: 0:13:31  model_time: 1.5155 (1.5876)  evaluator_time: 0.9315 (0.9757)  time: 2.